# Heart Dataset

**Raihana Fawaz (1103210102)**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
# Memuat dataset
df = pd.read_csv('heart.csv')

df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
# Praproses Data: Pisahkan fitur dan target
X = df.drop('target', axis=1).values
y = df['target'].values

In [6]:
# Normalisasi data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
# Mengonversi data ke dalam bentuk tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [9]:
# Membuat model MLP
def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    # Lapisan input ke lapisan tersembunyi pertama
    layers.append(nn.Linear(input_size, hidden_neurons))

    # Menambahkan lapisan tersembunyi tambahan
    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))

    # Memilih fungsi aktivasi
    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()

    # Menambahkan lapisan output
    layers.append(nn.Linear(hidden_neurons, 2))  # Menghasilkan 2 kelas (0 atau 1)
# Membuat model MLP
def create_mlp(input_size, hidden_layers, hidden_neurons, activation_function):
    layers = []
    # Lapisan input ke lapisan tersembunyi pertama
    layers.append(nn.Linear(input_size, hidden_neurons))

    # Menambahkan lapisan tersembunyi tambahan
    for _ in range(hidden_layers - 1):
        layers.append(nn.Linear(hidden_neurons, hidden_neurons))

    # Memilih fungsi aktivasi
    if activation_function == 'linear':
        activation = nn.Identity()
    elif activation_function == 'Sigmoid':
        activation = nn.Sigmoid()
    elif activation_function == 'ReLU':
        activation = nn.ReLU()
    elif activation_function == 'Softmax':
        activation = nn.Softmax(dim=1)
    elif activation_function == 'Tanh':
        activation = nn.Tanh()

    # Menambahkan lapisan output
    layers.append(nn.Linear(hidden_neurons, 2))  # Menghasilkan 2 kelas (0 atau 1)

    # Menggabungkan semua lapisan menjadi satu model
    model = nn.Sequential(*layers)
    return model

In [10]:
# Fungsi untuk melatih dan mengevaluasi model
def train_and_evaluate(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate):
    # DataLoader untuk pelatihan dalam batch
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Optimizer (Adam) dan fungsi loss (CrossEntropyLoss untuk klasifikasi)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Loop pelatihan
    model.train()  # Mengatur model ke mode pelatihan
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Mengatur ulang gradien
            output = model(X_batch)  # Melakukan prediksi dengan model
            loss = criterion(output, y_batch)  # Menghitung nilai loss
            loss.backward()  # Melakukan backpropagation
            optimizer.step()  # Memperbarui parameter model

    # Evaluasi model setelah pelatihan
    model.eval()  # Mengatur model ke mode evaluasi (tanpa pembaruan gradien)
    with torch.no_grad():  # Menonaktifkan perhitungan gradien untuk efisiensi
        # Akurasi pelatihan
        train_output = model(X_train_tensor)
        train_pred = torch.argmax(train_output, dim=1)  # Prediksi kelas menggunakan argmax
        train_accuracy = accuracy_score(y_train_tensor.numpy(), train_pred.numpy())  # Menghitung akurasi pelatihan
        train_loss = criterion(train_output, y_train_tensor).item()  # Menghitung loss pada data pelatihan

        # Akurasi pengujian
        test_output = model(X_test_tensor)
        test_pred = torch.argmax(test_output, dim=1)
        test_accuracy = accuracy_score(y_test_tensor.numpy(), test_pred.numpy())  # Menghitung akurasi pengujian
        test_loss = criterion(test_output, y_test_tensor).item()  # Menghitung loss pada data pengujian

    return train_accuracy, train_loss, test_accuracy, test_loss  # Mengembalikan hasil

In [11]:
# Daftar hyperparameter untuk eksperimen
hidden_layers_options = [1, 2, 3]  # Jumlah lapisan tersembunyi
hidden_neurons_options = [8, 16, 32]  # Jumlah neuron di setiap lapisan tersembunyi
activation_functions = ['linear', 'Sigmoid', 'ReLU', 'Softmax', 'Tanh']  # Fungsi aktivasi yang digunakan
epochs_options = [10, 25, 50, 100]  # Jumlah epoch pelatihan
learning_rates = [0.1, 0.01, 0.001, 0.0001]  # Tingkat pembelajaran
batch_sizes = [64, 128, 256, 512]  # Ukuran batch data

# Melakukan eksperimen dengan semua kombinasi hyperparameter
results = []  # Menyimpan hasil eksperimen
best_result = None
worst_result = None

for hidden_layers in hidden_layers_options:
    for hidden_neurons in hidden_neurons_options:
        for activation_function in activation_functions:
            for epochs in epochs_options:
                for learning_rate in learning_rates:
                    for batch_size in batch_sizes:
                        # Mencetak konfigurasi hyperparameter yang sedang digunakan
                        print(f"Melatih dengan {hidden_layers} lapisan tersembunyi, {hidden_neurons} neuron, fungsi aktivasi {activation_function}, "
                              f"{epochs} epoch, tingkat pembelajaran {learning_rate}, ukuran batch {batch_size}")

                        # Membuat model dengan kombinasi hyperparameter tertentu
                        model = create_mlp(X_train_tensor.shape[1], hidden_layers, hidden_neurons, activation_function)

                        # Melatih dan mengevaluasi model
                        train_acc, train_loss, test_acc, test_loss = train_and_evaluate(
                            model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size, epochs, learning_rate
                        )

                        # Menyimpan hasil eksperimen
                        results.append({
                            'Hidden Layers': hidden_layers,
                            'Hidden Neurons': hidden_neurons,
                            'Activation Function': activation_function,
                            'Epochs': epochs,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size,
                            'Train Accuracy': train_acc,
                            'Train Loss': train_loss,
                            'Test Accuracy': test_acc,
                            'Test Loss': test_loss
                        })

                        # Mencetak hasil akhir dari eksperimen saat ini
                        print(f"Akurasi Pelatihan: {train_acc * 100:.2f}%")
                        print(f"Loss Pelatihan: {train_loss:.4f}")
                        print(f"Akurasi Pengujian: {test_acc * 100:.2f}%")
                        print(f"Loss Pengujian: {test_loss:.4f}\n")

                        # Memperbarui hasil terbaik dan terburuk
                        if best_result is None or test_acc > best_result['Test Accuracy']:
                            best_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }
                        if worst_result is None or test_acc < worst_result['Test Accuracy']:
                            worst_result = {
                                'Hyperparameters': {
                                    'Hidden Layers': hidden_layers,
                                    'Hidden Neurons': hidden_neurons,
                                    'Activation Function': activation_function,
                                    'Epochs': epochs,
                                    'Learning Rate': learning_rate,
                                    'Batch Size': batch_size
                                },
                                'Test Accuracy': test_acc
                            }

Streaming output truncated to the last 5000 lines.
Loss Pengujian: 0.6381

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi Sigmoid, 100 epoch, tingkat pembelajaran 0.0001, ukuran batch 512
Akurasi Pelatihan: 65.97%
Loss Pelatihan: 0.6207
Akurasi Pengujian: 64.61%
Loss Pengujian: 0.6163

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi ReLU, 10 epoch, tingkat pembelajaran 0.1, ukuran batch 64
Akurasi Pelatihan: 85.22%
Loss Pelatihan: 0.3720
Akurasi Pengujian: 81.82%
Loss Pengujian: 0.4054

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi ReLU, 10 epoch, tingkat pembelajaran 0.1, ukuran batch 128
Akurasi Pelatihan: 86.75%
Loss Pelatihan: 0.3403
Akurasi Pengujian: 80.19%
Loss Pengujian: 0.4287

Melatih dengan 3 lapisan tersembunyi, 8 neuron, fungsi aktivasi ReLU, 10 epoch, tingkat pembelajaran 0.1, ukuran batch 256
Akurasi Pelatihan: 86.19%
Loss Pelatihan: 0.3394
Akurasi Pengujian: 79.87%
Loss Pengujian: 0.4536

Melatih dengan 3 lapisan tersem

In [12]:
# Menampilkan best dan worst hyperparameter berdasarkan akurasi
print("\nBest Hyperparameter Configuration:")
print(f"Hidden Layers: {best_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {best_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {best_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {best_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {best_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {best_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {best_result['Test Accuracy'] * 100:.2f}%")


Best Hyperparameter Configuration:
Hidden Layers: 3
Hidden Neurons: 32
Activation Function: Softmax
Epochs: 100
Learning Rate: 0.0001
Batch Size: 256
Test Accuracy: 84.09%


In [13]:
# Menampilkan hyperparameter terburuk berdasarkan akurasi terendah
print("\nWorst Hyperparameter Configuration:")
print(f"Hidden Layers: {worst_result['Hyperparameters']['Hidden Layers']}")
print(f"Hidden Neurons: {worst_result['Hyperparameters']['Hidden Neurons']}")
print(f"Activation Function: {worst_result['Hyperparameters']['Activation Function']}")
print(f"Epochs: {worst_result['Hyperparameters']['Epochs']}")
print(f"Learning Rate: {worst_result['Hyperparameters']['Learning Rate']}")
print(f"Batch Size: {worst_result['Hyperparameters']['Batch Size']}")
print(f"Test Accuracy: {worst_result['Test Accuracy'] * 100:.2f}%")


Worst Hyperparameter Configuration:
Hidden Layers: 2
Hidden Neurons: 8
Activation Function: linear
Epochs: 10
Learning Rate: 0.0001
Batch Size: 256
Test Accuracy: 32.14%
